In [1]:
import json
import re
import pandas as pd
import numpy as np

In [2]:
#Reading data
with open("input1.json") as f:
    data = json.loads(f.read())

In [3]:
#url regular expression
urlRE = re.compile(r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))")

In [4]:
#accessing nested
#1st JSON object
def accessJSON(data):
    global c
    for key,value in data.items():
        if(type(value)==type(str())  or type(value)==type(int())):
            stack.append(key)
            stack.append(value)
            if(key.lower()=="dt"):
                i=stack.pop()
                if(i.lower()=="int" or i.lower() == "dec" or i.lower() =="str" or i.lower()=="bool" or i.lower() == "integer" or i.lower() =="decimal"  or i.lower() =="string" or i.lower()=="boolean"):
                    if(i.lower()=="int" or i.lower()=="integer"):
                        i="xsd:integer"
                    elif(i.lower()=="dec" or i.lower() == "decimal"):
                        i="xsd:decimal"
                    elif(i.lower()=="bool" or i.lower() == "boolean"):
                        i="xsd:boolean"
                    else:
                        i="xsd:string"
                elif(re.search(urlRE,i.lower())):
                        res = re.findall(r'#',i) 
                        i="xsd:"+i[(i.index('#')+1):]
                stack.append(i)

        if type(value) == type(dict()):
            accessJSON(value)
        elif type(value) == type(list()):
            c=c+1
            if(c==2):
                break
            for val in value:
                if type(val) == type(str()):
                    pass
                elif type(val) == type(list()):
                    pass
                else:
                    accessJSON(val)

In [5]:
def seperatekeyvalues(stackelements):
    keys = []
    values = []

    for i in range(len(stackelements)):
        if(i%2==0):
            keys.append(stackelements[i])
        else:
            values.append(stackelements[i])
    return keys, values

In [6]:
def comparekeys(w1,w2):
    if(w1==w2):
        return True
    else:
        return False

In [7]:
def findingrowscolumns(keys):
    start=''
    ei=0
    count = 0
    for i in range(len(keys)):
        if(start==''):
            start=keys[i]
            si=i
        elif(keys[i]==start):
            end=keys[i-1]
            ei=i-1
            break   

    j=0
    count = 0
    for i in range(len(keys)):
        if(j >= len(keys)):
            break
        for k in range(ei+1):
            if(comparekeys(keys[k],keys[k+ei+1])==False):
                print("Keys are not of same length")
        j=j+ei+1
        count=count+1
    return count, (ei+1)

In [8]:
def writingintoFile(fileName, rows, column,keys, values):
    a=np.array(values).reshape(rows, column)
    # convert array into dataframe 
    DF = pd.DataFrame(a, columns=keys[0:column]) 
    # save the dataframe as a csv file 
    DF.to_csv(fileName,index = False)

In [9]:
def accessJSONLT(data):
    global c
    for key,value in data.items():       
        if(type(value) == type(str()) and c==2):
            stackLG.append(key)
            stackLG.append(value)
        if type(value) == type(dict()):
            accessJSONLT(value)
        elif type(value) == type(list()):
            c=c+1
            if(c == 2):
                if(key.lower()!="languages"):
                    print("Error! Required language tag here!")
            for val in value:
                if type(val) == type(str()):
                    pass
                elif type(val) == type(list()):
                    pass
                else:
                    accessJSONLT(val)


In [19]:
def mergeDataset():
    df1 = pd.read_csv("data1.csv")
    header1=df1.columns.tolist()

    df2 = pd.read_csv("data2.csv")
    header2=df2.columns.tolist()

    count=0
    matchedHeader = []
    nextHeader = []

    for i in header1:
        for j in range(len(header2)):
            if(i==header2[j]):
                matchedHeader.append(i)
                count=count+1
                nextHeader.append(header2[j+1])
            if(count>=2):
                break

    res = df1.merge(df2, how='inner', left_on=[matchedHeader[0]], right_on=[matchedHeader[0]])

    return res

In [17]:
def changeLanguagetag(res):
    lTags = pd.read_csv("ltgs.csv")

    header=res.columns.tolist()

    langColumn = "language"
    for i in (header):
        if header=="lang":
           langcolumn = "lang"
           break

    languagePresent=res[[langColumn]].squeeze()

    langNew = []

    for i in range(len(languagePresent)):
        for j in range(len(lTags)):
            lp=str(languagePresent[i])
            if(str(lTags.loc[j,"Language"]).lower() == lp.lower()):
                languagePresent[i] = (lTags.loc[j,"tag"])

    res[[langColumn]]=languagePresent

    return res

In [22]:
stack=[]
c=0
accessJSON(data) #preprocess the datatype to required format
k,v=seperatekeyvalues(stack)
rows,columns=findingrowscolumns(k)
writingintoFile("data1.csv", rows, columns, k, v)   #writes datatype modified column to file

flag=0
#checking for language tag if any as another json object
stackLG=[]
c=0
accessJSONLT(data)
if(stackLG):
    k,v=seperatekeyvalues(stackLG)
    rows,columns=findingrowscolumns(k)
    writingintoFile("data2.csv", rows, columns, k, v)   #writes language tags to another file
    flag=1    

if(flag==1):
    finalDF=mergeDataset()
else:
    finalDF = pd.read_csv("data1.csv")

finalDF = changeLanguagetag(finalDF)


finalDF.to_csv("cleanedData.csv")